# Mock

### Topics

- What is Mock?
- use Mock to imitate objects in your tests
- check usage data to understand how you use your objects
- customize your mock objects' return values and side effects
- patch() objects through out your codebase
- common mock problems and avoiding the same

## Imitating objects using Mocks

- based on: https://realpython.com/python-mock-library/ and https://docs.python.org/3/library/unittest.mock.html

- isolated problems are easier to diagnose and solve
- if a test fails, diagnosing/debugging many interrelated components can be very difficult
    - e.g., why the engine of a gasoline car not firing
- we often want to isolate items and testing environments by providing simiplified imitations (facking real objects)
- two reasons to replace actual code/API with imitation or **mock/fake** objects
    1. isolate a unit under test - create collaborating classes and function so we can test one unknown component
    2. test code that requires an object that is either expensive or risky to use; things like shared databases, filesystems, cloud infrastructures can be be very expensive to setup and tear down for testing
    
- unittest.mock module provides Mock base class for mocking objects
- you can pass mock object as arguments to functions
- assign/patch other objects
- when substituting an object in your code, the Mock must look like the real object it is replacing
    - e.g., if you're mocking `json` library and your program calls `dumps()`, then the mock object must also contain `dumps()` 
    
- Mock must simulate any object that it replaces
    - Mock creates attributes/members when you access them!
- Mock methods can take whaterver arguments you provide but always return Mock object

In [9]:
from unittest.mock import Mock

In [155]:
help(Mock)

Help on class Mock in module unittest.mock:

class Mock(CallableMixin, NonCallableMock)
 |  Mock(*args, **kw)
 |  
 |  Create a new `Mock` object. `Mock` takes several optional arguments
 |  that specify the behaviour of the Mock object:
 |  
 |  * `spec`: This can be either a list of strings or an existing object (a
 |    class or instance) that acts as the specification for the mock object. If
 |    you pass in an object then a list of strings is formed by calling dir on
 |    the object (excluding unsupported magic attributes and methods). Accessing
 |    any attribute not in this list will raise an `AttributeError`.
 |  
 |    If `spec` is an object (rather than a list of strings) then
 |    `mock.__class__` returns the class of the spec object. This allows mocks
 |    to pass `isinstance` tests.
 |  
 |  * `spec_set`: A stricter variant of `spec`. If used, attempting to *set*
 |    or get an attribute on the mock that isn't on the object passed as
 |    `spec_set` will raise an `A

In [10]:
mock = Mock()

In [11]:
mock

<Mock id='140202251363488'>

In [12]:
mock.some_attribute

<Mock name='mock.some_attribute' id='140202251361472'>

In [13]:
mock.do_something()
# mock methods return Mock object

<Mock name='mock.do_something()' id='140202251365264'>

In [143]:
# let's use this Hello class to use some mocking/patching concepts
class Hello(object):
    def __init__(self, msg="Hello there") -> None:
        self.__msg = msg
        
    def greet(self) -> str:
        return self.__msg

In [144]:
hi = Hello()

In [145]:
print(hi.greet())

Hello there


In [146]:
mock_hi = Mock()

In [147]:
mock_hi.greet.return_value = 'Hello there'

In [148]:
mock_hi.greet()

'Hello there'

In [150]:
type(mock_hi)

unittest.mock.Mock

In [151]:
type(mock_hi.greet)

unittest.mock.Mock

In [152]:
assert mock_hi.greet() == 'Hello there'

In [153]:
mock_hi.greet.assert_called()

In [154]:
mock_hi.greet.call_count

2

In [137]:
# let's see Python std json library
import json

In [138]:
help(json)

Help on package json:

NAME
    json

MODULE REFERENCE
    https://docs.python.org/3.10/library/json.html
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    JSON (JavaScript Object Notation) <https://json.org> is a subset of
    JavaScript syntax (ECMA-262 3rd edition) used as a lightweight data
    interchange format.
    
    :mod:`json` exposes an API familiar to users of the standard library
    :mod:`marshal` and :mod:`pickle` modules.  It is derived from a
    version of the externally maintained simplejson library.
    
    Encoding basic Python object hierarchies::
    
        >>> import json
        >>> json.dumps(['foo', {'bar': ('baz', None, 1.0, 2)}])
        '["foo", {"bar": ["baz", nul

In [140]:
# dump requires two arguments
data = json.dump()

TypeError: dump() missing 2 required positional arguments: 'obj' and 'fp'

In [141]:
# patching json object
json = Mock()

In [142]:
json.dump()
# takes any or no arguments; returns Mock object

<Mock name='mock.dump()' id='140202278305312'>

### Assertions and Inspection

- Mock instances store data on how you used them
    - e.g., if you called a method, stores information on how you called the method, and so on
- examples of how to use this information

In [18]:
from unittest.mock import Mock

In [19]:
# create a mock object
json = Mock()

In [20]:
json.loads('{"key": "value"}')

<Mock name='mock.loads()' id='140202253942992'>

In [21]:
# we know that we called loads() 
# so we can make assertions to test that expectation
json.loads.assert_called()

In [22]:
json.loads.assert_called_once()

In [23]:
json.loads.assert_called_with('{"key": "value"}')

In [24]:
# this should raise an exception - AssertionError
json.loads.assert_called_with('{"key1": "value1"}')

AssertionError: expected call not found.
Expected: loads('{"key1": "value1"}')
Actual: loads('{"key": "value"}')

In [25]:
json.loads.assert_called_once_with('{"key": "value"}')

In [26]:
json.loads('{"key": "value"}')

<Mock name='mock.loads()' id='140202253942992'>

In [27]:
json.loads.assert_called_once()

AssertionError: Expected 'loads' to have been called once. Called 2 times.
Calls: [call('{"key": "value"}'), call('{"key": "value"}')].

In [28]:
json.loads.assert_not_called()

AssertionError: Expected 'loads' to not have been called. Called 2 times.
Calls: [call('{"key": "value"}'), call('{"key": "value"}')].

### Mock speical attributes

- helps you understand how your application used an object

In [29]:
from unittest.mock import Mock

In [30]:
json = Mock()

In [31]:
json.loads('{"key": "value"}')

<Mock name='mock.loads()' id='140202277001424'>

In [32]:
json.loads('{"key1": "value1"}')

<Mock name='mock.loads()' id='140202277001424'>

In [33]:
# number of times you called loads
json.loads.call_count

2

In [34]:
# the last loads call
json.loads.call_args

call('{"key1": "value1"}')

In [35]:
# list of calls to json's methods (recursively)
json.method_calls

[call.loads('{"key": "value"}'), call.loads('{"key1": "value1"}')]

### Managing a Mock's Return Value

- mocks let you control your code's behavior during tests
- one important aspect of the testing is to control object's behavior (methods) and their return values

- e.g., write a function that determines whether today is a weekday
- you can mock `datetime` and set the `.return_value` for `.today()` to a day that you choose

In [74]:
from datetime import datetime

In [75]:
datetime.today()

datetime.datetime(2023, 1, 5, 19, 43, 17, 17093)

In [76]:
def is_weekday():
    today = datetime.today()
    # Python's datetime library treats Monday as 0 and Sunday as 6
    return 0 <= today.weekday() < 5


In [77]:
# Test if today is a weekday
assert is_weekday()
# if you run the test on weekend, you'll get an AssertionError

In [78]:
tuesday = datetime(year=2023, month=1, day=3)
saturday = datetime(year=2023, month=1, day=7)

In [79]:
# Mock datetime to control today's date
datetime = Mock()

In [80]:
def is_weekday():
    today = datetime.today()
    # Python's datetime library treats Monday as 0 and Sunday as 6
    return 0 <= today.weekday() < 5

In [81]:
# Mock .today() to return Tuesday
datetime.today.return_value = tuesday

In [82]:
datetime.today.return_value

datetime.datetime(2023, 1, 3, 0, 0)

In [83]:
datetime.today().day

3

In [84]:
# Test tuesday is a weekday
assert is_weekday()

In [85]:
# Mock .today() to return Saturday
datetime.today.return_value = saturday

In [86]:
# Test Saturday is not a weekday
assert is_weekday()

AssertionError: 

### Managing Mock's side effects

- you can control code's behavior by specifing a mocked function's side effects (https://docs.python.org/3/library/unittest.mock.html#unittest.mock.Mock.side_effect)
- side effect defines what happens when you call the mocked function
- you can use side effect to mock the return values, e.g.
- see `ch_04_02/mock_demo.py` file for a full demo

In [49]:
mock = Mock()

In [50]:
# using side_effect to raise an exception
mock.side_effect = Exception('Boom')

In [51]:
mock()

Exception: Boom

In [52]:
# using side_effect to return a sequence of values
mock = Mock()
mock.side_effect = [3, 2, 1]

In [53]:
# call mock three times to return each side effect
mock(), mock(), mock()

(3, 2, 1)

### Configuring Your Mock

- can configure Mock to setup some the object's behaviors and attributes
- two ways to configure Mock:
    - when you create it or when you use `.configure_mock()` method

In [54]:
mock = Mock(side_effect=Exception)

In [55]:
mock()

Exception: 

In [56]:
mock = Mock(name='Mocking with Python')

In [57]:
mock.name

<Mock name='Mocking with Python.name' id='140202276995904'>

In [58]:
mock = Mock(return_value=True)

In [59]:
mock()

True

In [60]:
# using .configure_mock()
mock = Mock()

In [61]:
mock.configure_mock(return_value=True)

In [62]:
mock()

True

In [63]:
mock = Mock(return_value='fish')

In [64]:
mock()

'fish'

## patching

- patching makes it easier to Mock objects that are imported from different module
- `unittest.mock` provides a powerful mechanism for mocking objects called `patch()`
- `patch()` looks up an object in a given module and replaces that object with a Mock
- usually, you use patch() as a decorator or a context manager to provide a scope in which you'll mock the target object
- patching a class replaces the class with `MagicMock` instance
- see these demos: `ch04_02/patch_demo.py` and `ch04_02/patch_demo1.py`

### patch( ) as a Decorator
- use patch() as a decorator to mock an object for the duration of your entire test function
- you can stack multiple patch decorators - see examples here: https://docs.python.org/3/library/unittest.mock.html

In [165]:
from unittest.mock import patch

In [303]:
@patch('__main__.Hello', spec=True)
def test_Hello(Hello_Mock):
    assert Hello_Mock is Hello
    # If the class is instantiated in the code under test 
    # then it will be the return_value of the mock that will be used.
    h = Hello_Mock.return_value
    h.greet.return_value = "Howdy!"
    assert h.greet() == 'Howdy!'
    assert Hello().greet() == 'Howdy!'
    assert Hello_Mock.called
    assert Hello.called

In [304]:
test_Hello()

### patch( ) as a Context Manager
- you only want to mock an object for a part of the test scope
- you're already using too many decorators or parameters which hurts your test readability


In [307]:
with patch('__main__.Hello', spec=True) as Hello_Mock:
    assert Hello_Mock is Hello
    h = Hello_Mock.return_value
    h.greet.return_value = "Good Bye!"
    assert h.greet() == 'Good Bye!'
    assert Hello().greet() == 'Good Bye!'
    assert Mock_Hello.called
    assert Hello.called

### patching only object's attributes/API

- you can mock only one method of an object instead of the entire object
- use `patch.object(class, 'class_method')`

In [156]:
from unittest.mock import patch

In [274]:
# just mock the greet method
@patch.object(Hello, 'greet', spec=True)
def test_greet(mock_greet) -> None:
    Hello.greet.return_value = 'Howdy!'
    # mock_greet is same as Hello.greet
    assert mock_greet() == 'Howdy!'


In [275]:
test_greet()

In [280]:
import requests
from requests.exceptions import Timeout

In [283]:
response = requests.get('https://example.com')

In [284]:
print(response)

<Response [200]>


In [287]:
response.status_code

200

In [286]:
help(response)

Help on Response in module requests.models object:

class Response(builtins.object)
 |  The :class:`Response <Response>` object, which contains a
 |  server's response to an HTTP request.
 |  
 |  Methods defined here:
 |  
 |  __bool__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if the status code of the response is between
 |      400 and 600 to see if there was a client error or a server error. If
 |      the status code, is between 200 and 400, this will return True. This
 |      is **not** a check to see if the response code is ``200 OK``.
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, *args)
 |  
 |  __getstate__(self)
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |      Allows you to use a response as an iterator.
 |  
 |  __nonzero__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if

In [290]:
@patch.object(requests, 'get') # (class, 'class_method')
def test_requests_get_404(mock_method):
    url = 'http://localhost/calendar/api/holidays'
    requests.get.return_value = Mock(status_code=404)
    response = requests.get(url)
    mock_method.assert_called_with(url)
    assert(response.status_code == 404)

In [291]:
test_requests_get()

### Patching Standard IO

- if the Class API/function uses print, stdin, it'll be difficult to unittest without patching stdio
- you can patch stdio with StringIO

In [310]:
def answer():
    print('Some Result')
    

In [311]:
from io import StringIO

In [319]:
@patch('sys.stdout', new_callable=StringIO)
def test_answer(mock_stdout) -> None:
    answer()
    assert mock_stdout.getvalue() == 'Some Result\n'

In [320]:
test_answer()

In [323]:
with patch('sys.stdout', new_callable=StringIO) as mock_stdout:
    answer()
    assert mock_stdout.getvalue() == 'Some Result\n'

In [369]:
# standard input
import sys

def getData():
    x = sys.stdin.read()
    return x

def getLines():
    x = sys.stdin.readlines()
    return x

def getInput():
    x = input()
    return x

In [370]:
data = getInput()

some data


In [371]:
data

'some data'

In [364]:
@patch('sys.stdin')
def test_getData(mock_stdin) -> None:
    mock_stdin.read.return_value = '2 3\n'
    data = getData()
    assert data == '2 3\n'

In [365]:
test_getData()

In [368]:
with patch('sys.stdin') as mock_stdin:
    mock_stdin.readlines.return_value = ['1 2\n', '3 4\n']
    data = getLines()
    print(data)
    assert data == ['1 2\n', '3 4\n']
    

['1 2\n', '3 4\n']


In [372]:
with patch('__main__.input') as mock_input:
    mock_input.return_value = '1 2 3 4'
    assert getInput() == '1 2 3 4'

### where to patch

- for patching to work, you must ensure that you patch the name used by the system under test
- must tell `patch()` correctly where to look for the object/name you want mocked
- if you choose the wrong target location, the result of `patch()` could be something you didn't expect
- good rule of thumb is to patch() the object where it is *looked up*

In [110]:
# datetime module is imported
import datetime
from unittest.mock import patch

In [112]:
with patch('datetime.datetime'):
    # datetime class is patched MagicMock now
    print(datetime)
    print(datetime.datetime)

<module 'datetime' from '/opt/anaconda3/envs/py/lib/python3.10/datetime.py'>
<MagicMock name='datetime' id='140202278341216'>


In [117]:
# just the datetime class is imported
from datetime import datetime

In [120]:
# the patch has no effect because there's 
# unmocked datetime class imported into the current scope
with patch('datetime.datetime'):
    print(datetime)
    print(datetime.today())

<class 'datetime.datetime'>
2023-01-05 20:37:26.930897


In [121]:
# if you need to patchdatetime class imported into the global namespace
# you do the following
with patch('__main__.datetime'):
    print(datetime)

<MagicMock name='datetime' id='140202253954032'>


### common mocking problems

### changes to object interfaces and misspellings

- when interface of an object changes, any tests relying on a Mock of that object may become irrelevant
- misspelling can break a test; recall that Mock creates its interface when you access its members
    - you'll essentially create a new interface when you misspell a name

### changes to external dependencies

- when external dependency changes its interface, your Python objects will become invalid
- your tests will pass but the actual production code will fail

### avoiding common problems using specifications

- use `spec` parameter providing the list of valid interface/method names of module/class you're mocking

In [122]:
from unittest.mock import Mock

In [123]:
# provide a list of valid api names of the class you want to mock 
calendar = Mock(spec=['is_weekday', 'get_holidays'])

In [124]:
calendar.is_weekday()

<Mock name='mock.is_weekday()' id='140202277534256'>

In [126]:
# Mock raises AttributeError
calendar.create_event()

AttributeError: Mock object has no attribute 'create_event'

In [129]:
# automatically create specification
from unittest.mock import create_autospec

from src.ch_04_02 import my_calendar

In [130]:
calendar = create_autospec(my_calendar)

In [133]:
calendar.is_weekday()

<MagicMock name='mock.is_weekday()' id='140202278122688'>

In [134]:
calendar.create_event()

AttributeError: Mock object has no attribute 'create_event'

In [135]:
# if you're using patch do the following
with patch('__main__.my_calendar', autospec=True) as calendar:
    calendar.is_weekday()
    calendar.create_event()

AttributeError: Mock object has no attribute 'create_event'

## How much testing is enough?

- how much is our code is actutally being tested
- According to E. W. Dijkstra - "Program testing can be used to show the presence of bugs, but never to show their absence!"
- **code coverage** is a count of the number of lines of code that are executed by a program and by test code
- we can download and use **coverage** library to do just that
- install coverage first

```bash
python -m pip install coverage

```bash
(py) ╭─rbasnet@MacBook-Pro ~/projects/Python-Object-Oriented-Programming/src/ch_04 ‹main●› 
╰─$ coverage run -m pytest tests/test_stats.py 
============================ test session starts =============================
platform darwin -- Python 3.10.8, pytest-7.1.2, pluggy-1.0.0
rootdir: /Users/rbasnet/projects/Python-Object-Oriented-Programming/src/ch_04
plugins: anyio-3.5.0
collected 3 items                                                            

tests/test_stats.py ...                                                [100%]

(py) ╭─rbasnet@MacBook-Pro ~/projects/Python-Object-Oriented-Programming/src/ch_04 ‹main●› 
╰─$ coverage report
Name                  Stmts   Miss  Cover
-----------------------------------------
stats.py                 20      0   100%
tests/__init__.py         0      0   100%
tests/test_stats.py      15      0   100%
-----------------------------------------

(py) ╭─rbasnet@MacBook-Pro ~/projects/Python-Object-Oriented-Programming/src/ch_04 ‹main●› 
╰─$ coverage report -m
Name                  Stmts   Miss  Cover   Missing
---------------------------------------------------
stats.py                 20      0   100%
tests/__init__.py         0      0   100%
tests/test_stats.py      15      0   100%
---------------------------------------------------
TOTAL                    35      0   100%

```

- **html** report gives more interactive report especially if the coverage is not 100%
```bash

```bash
(py) ╭─rbasnet@MacBook-Pro ~/projects/Python-Object-Oriented-Programming/src/ch_04_02 ‹main●› 
╰─$ coverage html
Wrote HTML report to htmlcov/index.html
```

## Exercises

- solve the following Kattis problems using OOD
- must write adequate unittesting for the class API
- must write integration testing using Mock to simulate input and output for complete program testing
- use coverage to create html coverage report of your testing

1. FizzBuzz - https://open.kattis.com/problems/fizzbuzz
2. Mixed Fractions - https://open.kattis.com/problems/mixedfractions
